In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array


trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)

df_data.head()
#df_data.describe()

testDataPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
df_test = pd.read_csv(testDataPath, header=0)

df_test.head()

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0


In [4]:
##Analyze the combinations of R and C in test, so that we can train based pn R and C values
count = 0
for group in df_data.groupby(["R", "C"]):
    print(len(group[1]))
    print(type(group[1]))
    data_inter = group[1].copy()
    print(data_inter.head())
    #print("print R=%s, C=%s" %(group[1]["R"], group[1]["C"]))

##We find that there are 9 unique R and C's for both train and test, 
## Craeting 9 different models with each data

664960
<class 'pandas.core.frame.DataFrame'>
        id  breath_id  R   C  time_step        u_in  u_out   pressure
2560  2561         39  5  10   0.000000   33.092845      0   6.189002
2561  2562         39  5  10   0.033633  100.000000      0   7.665347
2562  2563         39  5  10   0.067378   82.875875      0  14.554958
2563  2564         39  5  10   0.101031   63.513762      0  19.476108
2564  2565         39  5  10   0.134777   55.044110      0  21.655474
662160
<class 'pandas.core.frame.DataFrame'>
        id  breath_id  R   C  time_step       u_in  u_out   pressure
1120  1121         17  5  20   0.000000  12.557928      0   6.681117
1121  1122         17  5  20   0.034231  27.125642      0   7.665347
1122  1123         17  5  20   0.069309  29.353279      0   9.211995
1123  1124         17  5  20   0.103377  31.612488      0  10.196225
1124  1125         17  5  20   0.137508  32.978720      0  11.602268
661680
<class 'pandas.core.frame.DataFrame'>
      id  breath_id  R   C  tim

In [8]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
##Use complete training data for trainig.

appendDict = data_inter.iloc[-1,:].to_dict()
data_inter = data_inter.append(appendDict, ignore_index=True)
data_inter["pressure"] = data_inter["pressure"].shift(periods=1, fill_value=0)
data_gen = TimeseriesGenerator(data_inter.iloc[:,2:-1], data_inter.iloc[:,-1], length=1, sampling_rate=1, stride=1, batch_size=80)

print("Done Timeseries Generation")
## Creating datasets from the groups



Done Timeseries Generation


In [9]:
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, GRU
##Create RNN for predictions
n_input = 1
n_features=5

##Clear models
tf.keras.backend.clear_session()

model = Sequential()
model.add(GRU(128, activation='relu', return_sequences=True, input_shape=(n_input, n_features)))
model.add(GRU(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 1, 128)            51840     
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 89,153
Trainable params: 89,153
Non-trainable params: 0
_________________________________________________________________


In [11]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True,\
                                             verbose=1)
history = model.fit_generator(data_gen, epochs = 10, steps_per_epoch=len(data_inter) / 80, callbacks=[earlystop])

Epoch 1/10
8189/8189 [==============================] - 212s 26ms/step - loss: 26.0001 - mean_absolute_error: 2.8145
Epoch 2/10
8189/8189 [==============================] - 211s 26ms/step - loss: 22.6587 - mean_absolute_error: 2.4705
Epoch 3/10
8189/8189 [==============================] - 213s 26ms/step - loss: 21.8337 - mean_absolute_error: 2.3829
Epoch 4/10
8189/8189 [==============================] - 209s 26ms/step - loss: 21.4935 - mean_absolute_error: 2.3575
Epoch 5/10
8189/8189 [==============================] - 211s 26ms/step - loss: 21.2809 - mean_absolute_error: 2.3309
Epoch 6/10
8189/8189 [==============================] - 215s 26ms/step - loss: 21.1469 - mean_absolute_error: 2.3137
Epoch 7/10
8189/8189 [==============================] - 211s 26ms/step - loss: 21.0450 - mean_absolute_error: 2.3065
Epoch 8/10
8189/8189 [==============================] - 212s 26ms/step - loss: 20.9479 - mean_absolute_error: 2.2990
Epoch 9/10
8189/8189 [==============================] - 211s 26m

In [13]:
print(data_inter.head())
model.save("./ventilator_rnn_128_64GRU_r50_c50.h5")

      id  breath_id     R     C  time_step      u_in  u_out  pressure
0  241.0        4.0  50.0  50.0   0.000000  0.000000    0.0  0.000000
1  242.0        4.0  50.0  50.0   0.031854  1.262539    0.0  0.000000
2  243.0        4.0  50.0  50.0   0.063761  4.001352    0.0  6.259305
3  244.0        4.0  50.0  50.0   0.096498  6.379411    0.0  5.767190
4  245.0        4.0  50.0  50.0   0.128229  8.295194    0.0  6.540513
